In [1]:
%pip install -q numpy pandas scipy scikit-learn torch tqdm gensim matplotlib mol2vec esm
%pip install -q rdkit psutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 138.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 k

In [2]:
!python -c "from esm.models.esmc import ESMC; print('ESM-C installed successfully!')"

ESM-C installed successfully!


In [3]:
import os
import pathlib
import subprocess

!git clone --branch new-prot-embs https://github.com/glucose20/Temp.git

Cloning into 'Temp'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 223 (delta 76), reused 154 (delta 58), pack-reused 45 (from 2)
Receiving objects: 100% (223/223), 94.48 MiB | 22.31 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [4]:
# Import library
import kagglehub

# Download dataset from Kaggle (LLMDTA dataset)
path = kagglehub.dataset_download("christang0002/llmdta")

!ls {path}/pretrain-feature/pretrained-feature

Using Colab cache for faster access to the 'llmdta' dataset.
davis  kiba  metz


In [5]:
import os
import shutil


source_dir1 = f'{path}/pretrain-feature/pretrained-feature/davis'
source_dir2 = f'{path}/pretrain-feature/pretrained-feature/kiba'
source_dir3 = f'{path}/pretrain-feature/pretrained-feature/metz'
destination_dir = '/content/Temp/data'

# Remove destination directories if they exist
if os.path.exists(os.path.join(destination_dir, os.path.basename(source_dir1))):
    shutil.rmtree(os.path.join(destination_dir, os.path.basename(source_dir1)))
if os.path.exists(os.path.join(destination_dir, os.path.basename(source_dir2))):
    shutil.rmtree(os.path.join(destination_dir, os.path.basename(source_dir2)))
if os.path.exists(os.path.join(destination_dir, os.path.basename(source_dir3))):
    shutil.rmtree(os.path.join(destination_dir, os.path.basename(source_dir3)))

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy the directory
shutil.copytree(source_dir1, os.path.join(destination_dir, os.path.basename(source_dir1)))
shutil.copytree(source_dir2, os.path.join(destination_dir, os.path.basename(source_dir2)))
shutil.copytree(source_dir3, os.path.join(destination_dir, os.path.basename(source_dir3)))

print(f"Copied {source_dir3} to {destination_dir}")

Copied /kaggle/input/llmdta/pretrain-feature/pretrained-feature/metz to /content/Temp/data


In [6]:
!tar -xzf /content/Temp/data/dta-5fold-dataset/davis.tar.gz -C /content/Temp/data/dta-5fold-dataset/
!tar -xzf /content/Temp/data/dta-5fold-dataset/kiba.tar.gz -C /content/Temp/data/dta-5fold-dataset/
!tar -xzf /content/Temp/data/dta-5fold-dataset/metz.tar.gz -C /content/Temp/data/dta-5fold-dataset/

In [7]:
%cd /content/Temp

/content/Temp


In [8]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

# ESM-C imports
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

In [9]:
# Select ESM-C model variant
MODEL_NAME = "esmc_6b"  # Options: esmc_300m, esmc_600m, esmc_6b (via Forge API)
USE_FORGE_API = True  # Set True for esmc_6b with Forge token

# Forge API configuration (only needed for esmc_6b)
FORGE_TOKEN = "7GPrbbYRP3JXI1JXYz1z2t"  # Get from https://forge.evolutionaryscale.ai

# Device configuration (for local models only)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Model dimensions
MODEL_DIMS = {
    "esmc_300m": 960,
    "esmc_600m": 1152,
    "esmc_6b": 2560
}
EMBEDDING_DIM = MODEL_DIMS[MODEL_NAME]
print(f"Model: {MODEL_NAME}, Embedding dimension: {EMBEDDING_DIM}")
print(f"Using Forge API: {USE_FORGE_API}")

Using device: cuda
Model: esmc_6b, Embedding dimension: 2560
Using Forge API: True


In [10]:
# Load model
if USE_FORGE_API:
    print(f"Connecting to ESM Forge API for {MODEL_NAME}...")
    from esm.sdk.forge import ESM3ForgeInferenceClient

    model = ESM3ForgeInferenceClient(
        model="esmc-6b-2024-12",
        url="https://forge.evolutionaryscale.ai",
        token=FORGE_TOKEN
    )
    print("Connected to Forge API successfully!")
else:
    print(f"Loading local model: {MODEL_NAME}...")
    model = ESMC.from_pretrained(MODEL_NAME).to(DEVICE)
    model.eval()  # Set to evaluation mode
    print("Model loaded successfully!")

Connecting to ESM Forge API for esmc_6b...
Connected to Forge API successfully!


In [11]:
def get_esmc_pretrain(model, df_dir, db_name, sep=' ', header=None,
                      col_names=['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label'],
                      use_forge_api=False, device='cuda'):
    """
    Extract ESM-C embeddings for protein sequences.

    Args:
        model: Loaded ESM-C model or Forge API client
        df_dir: Path to input CSV file
        db_name: Database name for output filename
        sep: CSV separator
        header: CSV header row (None for no header)
        col_names: Column names for the dataframe
        use_forge_api: Whether using Forge API for esmc_6b
        device: Device for computation ('cuda' or 'cpu')

    Returns:
        Dictionary containing embeddings
    """
    # Load data
    df = pd.read_csv(df_dir, sep=sep, header=header)
    df.columns = col_names
    df.drop_duplicates(subset='prot_id', inplace=True)

    prot_ids = df['prot_id'].tolist()
    prot_seqs = df['prot_seq'].tolist()

    emb_dict = {}
    emb_mat_dict = {}
    length_dict = {}

    print(f"Processing {len(prot_ids)} proteins on {device}...")

    if use_forge_api:
        # Use Forge Batch Executor for efficient API calls
        from esm.sdk import batch_executor

        def embed_sequence(client, sequence, prot_id):
            """Helper function for batch processing"""
            try:
                seq = sequence[:2048]
                protein = ESMProtein(sequence=seq)
                protein_tensor = client.encode(protein)

                from esm.sdk.api import ESMProteinError
                if isinstance(protein_tensor, ESMProteinError):
                    raise protein_tensor

                output = client.logits(
                    protein_tensor,
                    LogitsConfig(sequence=True, return_embeddings=True)
                )

                # CRITICAL: Convert BFloat16 immediately after receiving from API
                embeddings = output.embeddings

                # Convert to torch tensor first if needed, then to float32
                if not isinstance(embeddings, torch.Tensor):
                    embeddings = torch.tensor(embeddings)

                # Convert BFloat16 to Float32
                if embeddings.dtype == torch.bfloat16:
                    embeddings = embeddings.to(torch.float32)

                # Now convert to numpy (Forge API runs on server, so CPU here is fine)
                if embeddings.is_cuda:
                    embeddings = embeddings.cpu()
                embeddings = embeddings.numpy()

                return (prot_id, embeddings, len(seq))
            except Exception as e:
                import traceback
                print(f"\nError processing {prot_id}: {e}")
                print(traceback.format_exc())
                return (prot_id, None, len(sequence[:2048]))

        # Process in batches using Forge executor
        print("Using Forge Batch Executor (server-side GPU processing)...")
        with batch_executor() as executor:
            # Prepare data for batch execution
            batch_data = [
                {'client': model, 'sequence': seq, 'prot_id': pid}
                for pid, seq in zip(prot_ids, prot_seqs)
            ]

            outputs = executor.execute_batch(
                user_func=embed_sequence,
                **{k: [d[k] for d in batch_data] for k in batch_data[0].keys()}
            )

        # Process outputs
        for prot_id, embeddings, seq_len in outputs:
            length_dict[prot_id] = seq_len

            if embeddings is not None:
                # Ensure float32 dtype
                if embeddings.dtype != np.float32:
                    embeddings = embeddings.astype(np.float32)

                # Store mean embedding as sequence representation
                emb_dict[prot_id] = embeddings.mean(axis=0)
                emb_mat_dict[prot_id] = embeddings
            else:
                # Fallback to zeros
                emb_dict[prot_id] = np.zeros(EMBEDDING_DIM, dtype=np.float32)
                emb_mat_dict[prot_id] = np.zeros((seq_len, EMBEDDING_DIM), dtype=np.float32)

    else:
        # Local model processing - USE GPU!
        print(f"Using local GPU: {device}")
        for idx in tqdm(range(len(prot_ids))):
            prot_id = str(prot_ids[idx])
            seq = prot_seqs[idx][:2048]
            length_dict[prot_id] = len(seq)

            try:
                # Create ESMProtein object
                protein = ESMProtein(sequence=seq)

                # Encode the protein (stays on GPU)
                protein_tensor = model.encode(protein)

                # Extract embeddings (computed on GPU)
                with torch.no_grad():
                    logits_output = model.logits(
                        protein_tensor,
                        LogitsConfig(return_embeddings=True)
                    )

                embeddings = logits_output.embeddings

                # Convert BFloat16 to Float32 if needed (still on GPU)
                if embeddings.dtype == torch.bfloat16:
                    embeddings = embeddings.to(torch.float32)

                # Only move to CPU at the last step before numpy conversion
                embeddings = embeddings.cpu().numpy()

                # Store mean embedding as sequence representation
                emb_dict[prot_id] = embeddings.mean(axis=0)  # Shape: (d_model,)

                # Store full embedding matrix
                emb_mat_dict[prot_id] = embeddings  # Shape: (seq_len, d_model)

            except Exception as e:
                print(f"\nError processing {prot_id}: {e}")
                # Use zero embeddings as fallback
                emb_dict[prot_id] = np.zeros(EMBEDDING_DIM, dtype=np.float32)
                emb_mat_dict[prot_id] = np.zeros((len(seq), EMBEDDING_DIM), dtype=np.float32)

    # Prepare output dictionary
    dump_data = {
        "dataset": db_name,
        "vec_dict": emb_dict,
        "mat_dict": emb_mat_dict,
        "length_dict": length_dict,
        "model": MODEL_NAME,
        "embedding_dim": EMBEDDING_DIM,
        "use_forge_api": use_forge_api
    }

    # Save to pickle
    output_file = f'./{db_name}_esmc_pretrain.pkl'
    with open(output_file, 'wb') as f:
        pickle.dump(dump_data, f)

    print(f"\nSaved embeddings to: {output_file}")
    print(f"Total proteins: {len(emb_dict)}")
    print(f"Embedding dimension: {EMBEDDING_DIM}")

    return dump_data


In [13]:
# Example: Process simple-Case study data
db_name = 'simple_case'
df_dir = r'/content/Temp/data/simple-Case/proteins.csv'
col_names = ['prot_id', 'prot_seq']

# Run the extraction (passes device to function)
result = get_esmc_pretrain(
    model=model,
    df_dir=df_dir,
    db_name=db_name,
    sep=',',
    header=0,
    col_names=col_names,
    use_forge_api=USE_FORGE_API,
    device=DEVICE  # Pass device config
)


Processing 4 proteins on cuda...
Using Forge Batch Executor (server-side GPU processing)...


Processing  100%|████████████████████████| 4/4 [Elapsed: 00:56 | Remaining: 00:00] , Success=4 Fail=0 Retry=0


Saved embeddings to: ./simple_case_esmc_pretrain.pkl
Total proteins: 4
Embedding dimension: 2560


In [12]:
# Dataset configurations
datasets = {
    # 'davis': './data/dta-5fold-dataset/davis/davis_prots.csv',
    'kiba': './data/dta-5fold-dataset/kiba/kiba_prots.csv',
    # 'metz': './data/dta-5fold-dataset/metz/metz_prots.csv'
}

col_names = ['prot_id', 'prot_seq']

# Process each dataset
for db_name, df_dir in datasets.items():
    print(f"\n{'='*60}")
    print(f"Processing {db_name.upper()} dataset")
    print(f"{'='*60}")

    try:
        result = get_esmc_pretrain(
            model=model,
            df_dir=df_dir,
            db_name=db_name,
            sep=',',  # Adjust if needed
            header=0,
            col_names=col_names,
            use_forge_api=USE_FORGE_API,
            device=DEVICE  # Pass device config
        )
        print(f"✓ Successfully processed {db_name}")
    except Exception as e:
        print(f"✗ Error processing {db_name}: {e}")



Processing KIBA dataset
Processing 229 proteins on cuda...
Using Forge Batch Executor (server-side GPU processing)...


Processing  100%|████████████████████████| 229/229 [Elapsed: 02:50 | Remaining: 00:00] , Success=229 Fail=0 Retry=0



Saved embeddings to: ./kiba_esmc_pretrain.pkl
Total proteins: 229
Embedding dimension: 2560
✓ Successfully processed kiba


In [ ]:
# Load and inspect generated embeddings
def verify_embeddings(db_name):
    file_path = f'./{db_name}_esmc_pretrain.pkl'

    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)

        print(f"\n{'='*60}")
        print(f"Verification: {db_name}")
        print(f"{'='*60}")
        print(f"Dataset: {data['dataset']}")
        print(f"Model: {data['model']}")
        print(f"Embedding dimension: {data['embedding_dim']}")
        print(f"Number of proteins: {len(data['vec_dict'])}")

        # Check first protein
        first_prot_id = list(data['vec_dict'].keys())[0]
        first_vec = data['vec_dict'][first_prot_id]
        first_mat = data['mat_dict'][first_prot_id]

        print(f"\nFirst protein: {first_prot_id}")
        print(f"  - Vec shape: {first_vec.shape}")
        print(f"  - Mat shape: {first_mat.shape}")
        print(f"  - Sequence length: {data['length_dict'][first_prot_id]}")

        return data
    except FileNotFoundError:
        print(f"✗ File not found: {file_path}")
        return None

# Verify each dataset
for db_name in ['davis', 'kiba', 'metz']:
    verify_embeddings(db_name)

In [ ]:
import os

file_names = ['davis_esmc_pretrain.pkl', 'kiba_esmc_pretrain.pkl', 'metz_esmc_pretrain.pkl', 'case_study_esmc_pretrain.pkl']
current_dir = os.getcwd()

print("Địa chỉ cụ thể của các file _esmc_pretrain.pkl:")
for file_name in file_names:
    full_path = os.path.join(current_dir, file_name)
    if os.path.exists(full_path):
        print(f"  - File '{file_name}' TỒN TẠI tại: {full_path}")
    else:
        print(f"  - File '{file_name}' KHÔNG TỒN TẠI tại: {full_path}")

In [15]:
import os
import shutil

file_suffixes = ['davis_esmc_pretrain.pkl', 'kiba_esmc_6b_pretrain.pkl', 'metz_esmc_pretrain.pkl', 'case_study_esmc_pretrain.pkl']
base_source_dir = '/content/Temp'
base_dest_dir = '/content/Temp/data'

print("Di chuyển các file _esmc_pretrain.pkl:")

for file_suffix in file_suffixes:
    dataset_name = file_suffix.split('_')[0] # e.g., 'davis' from 'davis_esmc_pretrain.pkl'
    source_path = os.path.join(base_source_dir, file_suffix)

    # Construct destination directory: /content/Temp/data/davis/
    destination_dataset_dir = os.path.join(base_dest_dir, dataset_name)
    os.makedirs(destination_dataset_dir, exist_ok=True)

    # Construct destination file name: /content/Temp/data/davis/davis_esm_pretrain.pkl
    # Note: The user requested _esm_pretrain.pkl, but files are _esmc_pretrain.pkl.
    # Keeping _esmc_pretrain.pkl for consistency with generated files.
    destination_file_name = f"{dataset_name}_esmc_pretrain.pkl"
    destination_path = os.path.join(destination_dataset_dir, destination_file_name)

    if os.path.exists(source_path):
        try:
            shutil.move(source_path, destination_path)
            print(f"  - Đã di chuyển '{file_suffix}' tới '{destination_path}'")
        except Exception as e:
            print(f"  - Lỗi khi di chuyển '{file_suffix}': {e}")
    else:
        print(f"  - File '{file_suffix}' KHÔNG TỒN TẠI tại '{source_path}', bỏ qua.")

print("Hoàn tất việc di chuyển file.")

# Verify new locations
print("\nKiểm tra lại địa chỉ mới của các file:")
for file_suffix in file_suffixes:
    dataset_name = file_suffix.split('_')[0]
    destination_file_name = f"{dataset_name}_esmc_pretrain.pkl"
    destination_path = os.path.join(base_dest_dir, dataset_name, destination_file_name)
    if os.path.exists(destination_path):
        print(f"  - File '{destination_file_name}' TỒN TẠI tại: {destination_path}")
    else:
        print(f"  - File '{destination_file_name}' KHÔNG TỒN TẠI tại: {destination_path}")

Di chuyển các file _esmc_pretrain.pkl:
  - File 'davis_esmc_pretrain.pkl' KHÔNG TỒN TẠI tại '/content/Temp/davis_esmc_pretrain.pkl', bỏ qua.
  - Đã di chuyển 'kiba_esmc_6b_pretrain.pkl' tới '/content/Temp/data/kiba/kiba_esmc_pretrain.pkl'
  - File 'metz_esmc_pretrain.pkl' KHÔNG TỒN TẠI tại '/content/Temp/metz_esmc_pretrain.pkl', bỏ qua.
  - File 'case_study_esmc_pretrain.pkl' KHÔNG TỒN TẠI tại '/content/Temp/case_study_esmc_pretrain.pkl', bỏ qua.
Hoàn tất việc di chuyển file.

Kiểm tra lại địa chỉ mới của các file:
  - File 'davis_esmc_pretrain.pkl' KHÔNG TỒN TẠI tại: /content/Temp/data/davis/davis_esmc_pretrain.pkl
  - File 'kiba_esmc_pretrain.pkl' TỒN TẠI tại: /content/Temp/data/kiba/kiba_esmc_pretrain.pkl
  - File 'metz_esmc_pretrain.pkl' KHÔNG TỒN TẠI tại: /content/Temp/data/metz/metz_esmc_pretrain.pkl
  - File 'case_esmc_pretrain.pkl' KHÔNG TỒN TẠI tại: /content/Temp/data/case/case_esmc_pretrain.pkl


In [ ]:
!pwd

/content/Temp


In [ ]:
!python code/train.py --fold 0 --cuda "0" --dataset davis --running_set warm --epochs 1 --batch_size 256

In [ ]:
!python regenerate_esmc_embeddings.py --dataset davis --model esmc_300m

Regenerating ESM-C embeddings for DAVIS
Model: esmc_300m (dim=960)
Device: cuda

Loading esmc_300m...
Fetching 4 files: 100% 4/4 [00:00<00:00, 5466.67it/s]
Model loaded!

Loading proteins from: ./data/dta-5fold-dataset/davis/davis_prots.csv
Found 442 unique proteins

Extracting embeddings...
100% 442/442 [02:23<00:00,  3.08it/s]

✅ Successfully saved to: ./data/davis/davis_esmc_pretrain.pkl
Total proteins: 442
Embedding dimension: 960

Verification:
  First protein: AAK1
  Vec shape: (963, 960) ❌
  Mat shape: (1, 963, 960) ❌



In [ ]:
!python code/train.py --fold 0 --cuda "0" --dataset davis --running_set warm --epochs 1 --batch_size 256

Training Fold 0/4
Dataset: davis-warm
Device: cuda (CUDA_VISIBLE_DEVICES=0)
Pretrain-./data/davis/davis_drug_pretrain.pkl
Pretrain-./data/davis/davis_esmc_pretrain.pkl
Loading fold 0 data...
  Train: ./data/dta-5fold-dataset/davis/warm/fold_0_train.csv
  Valid: ./data/dta-5fold-dataset/davis/warm/fold_0_valid.csv
  Test:  ./data/dta-5fold-dataset/davis/warm/fold_0_test.csv
Dataset loaded: 19236 train, 4809 valid, 6011 test samples
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/Temp/code/MyDataset.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drug_id, prot_id, label = pair[-3], pair[-2], pair[-1]
Traing Log

In [ ]:
!python code/train.py --fold 0 --cuda "0" --dataset kiba --running_set warm --epochs 1 --batch_size 256

Training Fold 0/4
Dataset: kiba-warm
Device: cuda (CUDA_VISIBLE_DEVICES=0)
Pretrain-./data/kiba/kiba_drug_pretrain.pkl
Pretrain-./data/kiba/kiba_esmc_pretrain.pkl
Loading fold 0 data...
  Train: ./data/dta-5fold-dataset/kiba/warm/fold_0_train.csv
  Valid: ./data/dta-5fold-dataset/kiba/warm/fold_0_valid.csv
  Test:  ./data/dta-5fold-dataset/kiba/warm/fold_0_test.csv
Dataset loaded: 75683 train, 18921 valid, 23650 test samples
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/Temp/code/MyDataset.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drug_id, prot_id, label = pair[-3], pair[-2], pair[-1]
Traing Log at fo

In [17]:
!python code/train.py --fold 0 --use_esmc true --esmc_model esmc_6b --epochs 1 --batch_size 128 --cuda 0 --dataset kiba


Training Fold 0/4
Dataset: kiba-novel-pair
ESM Model: ESM-C-esmc_6b (dim=2560)
Device: cuda (CUDA_VISIBLE_DEVICES=0)
Pretrain-./data/kiba/kiba_drug_pretrain.pkl
Pretrain-./data/kiba/kiba_esmc_6b_pretrain.pkl
Loading fold 0 data...
  Train: ./data/dta-5fold-dataset/kiba/novel-pair/fold_0_train.csv
  Valid: ./data/dta-5fold-dataset/kiba/novel-pair/fold_0_valid.csv
  Test:  ./data/dta-5fold-dataset/kiba/novel-pair/fold_0_test.csv
Dataset loaded: 40301 train, 57463 valid, 20490 test samples
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/Temp/code/MyDataset.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drug_id,